**用作协程的生成器**

In [1]:
import inspect


def simple_coro2(a):
    print('-> Started: a =', a)
    b = yield a
    print('-> Received: b =', b)
    c = yield a + b
    print('-> Received: c =', c)

In [2]:
s = simple_coro2(10)
# 检查协程状态
inspect.getgeneratorstate(s) 

'GEN_CREATED'

In [3]:
next(s)
inspect.getgeneratorstate(s)

-> Started: a = 10


'GEN_SUSPENDED'

In [4]:
s.send(12)
inspect.getgeneratorstate(s)

-> Received: b = 12


'GEN_SUSPENDED'

In [5]:
s.send(15)
inspect.getgeneratorstate(s)

-> Received: c = 15


StopIteration: 

In [6]:
inspect.getgeneratorstate(s)

'GEN_CLOSED'

**协程计算移动平均值**

In [17]:
from functools import wraps

# 预激装饰器，使得协程实例化后自动到第一个yield
def coroutine(func):
    """装饰器： 向前执行到第一个`yield`表达式， 预激`func`"""
    @wraps(func)
    def primer(*args,**kwargs): 
        gen = func(*args,**kwargs) 
        next(gen) 
        return gen 
    return primer

In [18]:
# 好处：使用函数的局部变量来保持上下文
@coroutine
def averager():
    total = 0.0
    count = 0
    average = None
#     无限循环，直到调用close或者引用被垃圾回收
    while True:
        term = yield average
        total += term
        count += 1
        average = total / count




In [19]:
a = averager()

In [20]:
a.send(10)

10.0

In [21]:
a.send(20)

15.0

In [22]:
a.close()

In [24]:
from collections import namedtuple
Result = namedtuple('Result', 'count average')

# 不在每次send时产出值，而是在传入指定参数时才用return
@coroutine
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break 
        total += term
        count += 1
        average = total/count
    return Result(count, average)

In [25]:
a = averager()

In [26]:
a.send(10)
a.send(20)
a.send(30)

In [27]:
try:
    a.send(None)
except StopIteration as e:
    result = e.value

In [28]:
result

Result(count=3, average=20.0)

In [1]:
import asyncio
import threading


In [ ]:
# 3.5以下
@asyncio.coroutine
def hello():
    print(threading.current_thread())
    yield from asyncio.sleep(5)
    print(threading.current_thread())


loop = asyncio.get_event_loop()
tasks = [hello(), hello()]
loop.run_until_complete(asyncio.wait(tasks))
loop.close()

In [ ]:
# 3.5以上
async def hello():
    print(threading.current_thread())
    await asyncio.sleep(5)
    print(threading.current_thread())


loop = asyncio.get_event_loop()
tasks = [hello(), hello()]
loop.run_until_complete(asyncio.wait(tasks))
loop.close()


In [ ]:
import asyncio


async def wget(host):
    print('Connect Host:%s' % host)
    connect = asyncio.open_connection(host, 80)
    reader, writer = await connect
    header = 'GET / HTTP/1.0\r\nHost: %s\r\n\r\n' % host
    writer.write(header.encode('utf-8'))
    await writer.drain()
    while True:
        line = await reader.readline()
        if line == b'\r\n':
            break
        print('%s header > %s' % (host, line.decode('utf-8').rstrip()))
    # Ignore the body, close the socket
    writer.close()


loop = asyncio.get_event_loop()
tasks = [wget(host) for host in ['www.baidu.com', 'www.sina.com']]
loop.run_until_complete(asyncio.wait(tasks))
loop.close()
